# Homework 18: Maximum Likelihood

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import utils as utils
from fractions import Fraction

from IPython.display import display, HTML

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

utils.load_custom_styles()

---
## Exercise 3.1

<img src="figures/homework-18/exercise-3.1.png" width="600" />




















We can compute the prior probability for each class $c_k$ as follows:

$$
P(c_k) = \frac{N_k}{\sum_l^K N_l}
$$
where $N_k$ is the number of samples in class $c_k$

Class $c_1$ has 7 samples and $c_2$ has 5 samples. In total, we have 12 samples. Therefore, $P(c_1) = 7/12=0.58$ and $P(c_2) = 5/12=0.42$.

In [12]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])

In [16]:
# Compute the mean vector associated with each class
m1 = np.mean(c1, axis=1).reshape(-1, 1)
m1

array([[-0.85714286],
       [-0.9       ]])

In [17]:
m2 = np.mean(c2, axis=1).reshape(-1, 1)
m2

array([[1.1],
       [1. ]])

Next, we compute:

$$
p(\mathbf{x} \mid c_1) = 
\frac
  { \exp(\lVert \mathbf{x} - \mu_i \rVert_2)  }
  {  \sum_{k=} }
$$

In [18]:
np.linalg(X - m1)

array([[ 0.85714286,  1.85714286, -0.14285714,  1.55714286,  0.65714286],
       [ 0.9       ,  1.9       ,  0.9       ,  0.7       ,  2.4       ]])

  ---
## Exercise 3.2

<img src="figures/homework-18/exercise-3.2.png" width="600" />




















---
## Exercise 3.3

<img src="figures/homework-18/exercise-3.3.png" width="600" />







---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.4a.png" width="600" />





<img src="figures/homework-18/exercise-3.4b.png" width="600" />














---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.5.png" width="600" />











